In [10]:
import findspark
findspark.init()
from pyspark.sql           import HiveContext
from pyspark.sql.functions import *
from pyspark.sql.types     import *
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [29]:
dfProduct = spark.read.csv(r"C:\Users\bwwil\Desktop\case\Case Impostos - Analytics - Data Engineer\Case Impostos - Analytics - Data Engineer\product.csv" ,header=True)
dfTax = spark.read.csv(r"C:\Users\bwwil\Desktop\case\Case Impostos - Analytics - Data Engineer\Case Impostos - Analytics - Data Engineer\tax.csv" ,header=True)
dfVol = spark.read.csv(r"C:\Users\bwwil\Desktop\case\Case Impostos - Analytics - Data Engineer\Case Impostos - Analytics - Data Engineer\vol_2019.csv.gz" ,header=True)

In [20]:
dfTax = dfTax.withColumn("month_year_join", concat(col("month_year").substr(5,4), lit("-"), col("month_year").substr(2,2), lit("-01")))
dfTax = dfTax.withColumn("tax", col("tax").cast(DecimalType(6, 2)))
dfTax = dfTax.groupBy("product", "month_year_join").agg(sum("tax").alias("tax"))
dfTax = dfTax.withColumnRenamed("product", "tax_product")

dfVol = dfVol.withColumn("customer", col("customer").cast("int")).fillna(0)
dfVol = dfVol.withColumn("plant", col("plant").cast("int")).fillna(0)
dfVol = dfVol.withColumn("volume", col("volume").cast(DecimalType(6, 2)))
dfVol = dfVol.groupBy("customer", "product", "month").agg(sum("volume").alias("volume"))

In [21]:
dfTax.show()
dfTax.count()

+-----------+---------------+---------+
|tax_product|month_year_join|      tax|
+-----------+---------------+---------+
|   84169688|     2019-01-01|    38.19|
|   67205397|     2019-01-01|    14.49|
|   67417001|     2019-01-01|  1594.84|
|   84160539|     2019-01-01|    17.24|
|   84166745|     2019-01-01|    31.04|
|      15284|     2019-02-01| -2346.44|
|   84169134|     2019-02-01|  7173.63|
|   34480624|     2019-02-01|     0.00|
|   84146085|     2019-02-01|  4173.93|
|   84171726|     2019-02-01|   835.08|
|   67417011|     2019-02-01|  2678.70|
|   67194485|     2019-02-01|   343.95|
|   67546017|     2019-02-01| -2830.55|
|   67578968|     2019-02-01| -5386.89|
|   67141547|     2019-02-01|    14.40|
|   67280879|     2019-03-01|-11859.83|
|   84169786|     2019-03-01|   191.30|
|   67511177|     2019-03-01|  5668.77|
|   67385668|     2019-03-01|     4.67|
|   84142684|     2019-03-01|    -6.74|
+-----------+---------------+---------+
only showing top 20 rows



20166

In [22]:
dfVol.show()
dfVol.count()

+--------+--------+----------+-------+
|customer| product|     month| volume|
+--------+--------+----------+-------+
|10007259|34480602|2019-01-01|   3.82|
|10007259|67318830|2019-01-01|   7.64|
|10007259|67643456|2019-01-01|  12.30|
|10007263|67417011|2019-01-01|   3.00|
|10007264|67369297|2019-01-01|   4.54|
|10007264|84156724|2019-01-01| 243.81|
|10007278|67138110|2019-01-01|  74.29|
|10007284|84153326|2019-01-01| 227.26|
|10007287|67142495|2019-01-01|1146.50|
|10007294|84157422|2019-01-01|   8.71|
|10007326|67309108|2019-01-01|  38.48|
|10007326|  942655|2019-01-01|  14.52|
|10007667|67294386|2019-01-01|  76.05|
|10007667|67541405|2019-01-01|  80.44|
|10007667|84171815|2019-01-01|  26.72|
|10007709|67276630|2019-01-01|   6.65|
|10007790|34480638|2019-01-01|   2.29|
|10007957|84144000|2019-01-01|   1.53|
|10007957|84161559|2019-01-01|  38.22|
|10008549|67294691|2019-01-01|   0.81|
+--------+--------+----------+-------+
only showing top 20 rows



3478259

In [23]:
cols = [ dfTax["tax_product"] == dfVol["product"], dfTax["month_year_join"] == dfVol["month"] ]
dfJoined = dfVol.join(dfTax, cols, "left").drop("tax_product", "month_year_join")
dfJoined.show()

+--------+--------+----------+-------+---------+
|customer| product|     month| volume|      tax|
+--------+--------+----------+-------+---------+
|10007259|34480602|2019-01-01|   3.82|    15.80|
|10007259|67318830|2019-01-01|   7.64|-14553.66|
|10007259|67643456|2019-01-01|  12.30| -4695.10|
|10007263|67417011|2019-01-01|   3.00|  3168.58|
|10007264|67369297|2019-01-01|   4.54|   635.37|
|10007264|84156724|2019-01-01| 243.81|  5956.31|
|10007278|67138110|2019-01-01|  74.29| -5981.08|
|10007284|84153326|2019-01-01| 227.26|  5433.25|
|10007287|67142495|2019-01-01|1146.50|  2829.42|
|10007294|84157422|2019-01-01|   8.71|    10.53|
|10007326|67309108|2019-01-01|  38.48| -6633.03|
|10007326|  942655|2019-01-01|  14.52| -3165.33|
|10007667|67294386|2019-01-01|  76.05|  3240.08|
|10007667|67541405|2019-01-01|  80.44|  3483.47|
|10007667|84171815|2019-01-01|  26.72|  7428.93|
|10007709|67276630|2019-01-01|   6.65|  3899.83|
|10007790|34480638|2019-01-01|   2.29|     7.34|
|10007957|84144000|2

In [26]:
dfJoined = dfJoined.withColumn("razao", (col("tax") / col("volume")).cast(DecimalType(6, 2)))
dfJoined.show()

+--------+--------+----------+-------+---------+--------+
|customer| product|     month| volume|      tax|   razao|
+--------+--------+----------+-------+---------+--------+
|10007259|34480602|2019-01-01|   3.82|    15.80|    4.14|
|10007259|67318830|2019-01-01|   7.64|-14553.66|-1904.93|
|10007259|67643456|2019-01-01|  12.30| -4695.10| -381.72|
|10007263|67417011|2019-01-01|   3.00|  3168.58| 1056.19|
|10007264|67369297|2019-01-01|   4.54|   635.37|  139.95|
|10007264|84156724|2019-01-01| 243.81|  5956.31|   24.43|
|10007278|67138110|2019-01-01|  74.29| -5981.08|  -80.51|
|10007284|84153326|2019-01-01| 227.26|  5433.25|   23.91|
|10007287|67142495|2019-01-01|1146.50|  2829.42|    2.47|
|10007294|84157422|2019-01-01|   8.71|    10.53|    1.21|
|10007326|67309108|2019-01-01|  38.48| -6633.03| -172.38|
|10007326|  942655|2019-01-01|  14.52| -3165.33| -218.00|
|10007667|67294386|2019-01-01|  76.05|  3240.08|   42.60|
|10007667|67541405|2019-01-01|  80.44|  3483.47|   43.31|
|10007667|8417

In [27]:
dfRazaoNeg = dfJoined.where(col("razao") < 0)
dfRazaoNeg.show()

+--------+--------+----------+------+---------+--------+
|customer| product|     month|volume|      tax|   razao|
+--------+--------+----------+------+---------+--------+
|10007259|67318830|2019-01-01|  7.64|-14553.66|-1904.93|
|10007259|67643456|2019-01-01| 12.30| -4695.10| -381.72|
|10007278|67138110|2019-01-01| 74.29| -5981.08|  -80.51|
|10007326|67309108|2019-01-01| 38.48| -6633.03| -172.38|
|10007326|  942655|2019-01-01| 14.52| -3165.33| -218.00|
|10007957|84161559|2019-01-01| 38.22| -8771.73| -229.51|
|10008560|67382771|2019-01-01|  2.55| -4913.59|-1926.90|
|10008560|67444311|2019-01-01|  3.92|-14825.81|-3782.09|
|10008605|84140606|2019-01-01|  6.88| -8010.96|-1164.38|
|10008718|67235818|2019-01-01|  2.37| -5809.73|-2451.36|
|10008740|67578976|2019-01-01|  3.02| -2173.81| -719.80|
|10008754|67643456|2019-01-01|  3.69| -4695.10|-1272.38|
|10008755|84165130|2019-01-01| 12.38| -2363.00| -190.87|
|10008814|84172474|2019-01-01| 28.66| -6962.36| -242.93|
|10009006|67501900|2019-01-01| 

In [28]:
dfRazaoPos = dfJoined.where(col("razao") > 23000)
dfRazaoPos.show()

+--------+-------+-----+------+---+-----+
|customer|product|month|volume|tax|razao|
+--------+-------+-----+------+---+-----+
+--------+-------+-----+------+---+-----+



In [45]:
dfProduct.show()
dfProduct.where(col("div") == "P").show()

+--------+---+--------+-----+------+----+---+----+----+
| product|div|category|brand|   cpg|bfpp|ppa| spf|spfv|
+--------+---+--------+-----+------+----+---+----+----+
|67313580|  F|      SA|   HL|F00663| 17S|54T|14WE|14WE|
|67313581|  F|      SA|   HL|F00663| 17S|54T|14WE|14WE|
|67313585|  F|      SA|   HL|F00663| 17S|54T|14WE|14WE|
|67313589|  F|      SA|   HL|F00663| 17S|54T|14WE|14WE|
|84172469|  F|      MA|   HL|F00478| 2AP|50Y|14W0|14W0|
|67174001|  F|      SE|   AS|F00054| 1ZG|82G|14J3|14J3|
|   15285|  F|      CS|   MZ|F01205| 1UB|6S5|14XE|14XE|
|   15287|  F|      CS|   MZ|F01205| 1UB|51P|14XF|14XF|
|   15289|  F|      CS|   MZ|F01205| 1UB|6UJ|14XD|14XD|
|   13997|  F|      OD|   KA|F01669| 1SC|51R|14WG|14WG|
|   13998|  F|      MA|   HL|F00478| 2AP|50Y|14W0|14W0|
|   14614|  F|      CS|   MZ|F01205| 1ZP|49P|14OU|14OU|
|   15284|  F|      CS|   MZ|F01205| 20U|51O|14RL|14RL|
|   15291|  F|      CS|   MZ|F01205| 20U|51O|14RL|14RL|
|   15292|  F|      CS|   MZ|F01205| 20U|7A0|14R